In [2]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os
from create_dataframe import *

In [7]:
final_df = pd.concat([pd.DataFrame(dataframe['filename']),pd.get_dummies(dataframe['label'])], axis=1)
final_df

In [8]:
import tensorflow as tf

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.,
            validation_split=0.20,
            rotation_range=90,
            horizontal_flip=True,
            vertical_flip=True,
            brightness_range=[0.3,1],
            )

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

In [51]:
img_dir = "./preprocessed_images"
img_size = 224
train_generator=train_datagen.flow_from_dataframe(
dataframe=dataframe,
directory=img_dir,
x_col="filename",
y_col="label",
subset="training",
batch_size=16,
seed=42,
shuffle=True,
class_mode="sparse",
target_size=(img_size,img_size))

valid_generator=train_datagen.flow_from_dataframe(
dataframe=dataframe,
directory=img_dir,
x_col="filename",
y_col="label",
subset="validation",
batch_size=16,
seed=42,
shuffle=True,
class_mode="sparse",
target_size=(img_size,img_size))

test_generator=test_datagen.flow_from_dataframe(
dataframe=dataframe,
directory=img_dir,
x_col="filename",
    
y_col="label",
batch_size=16,
shuffle=False,
class_mode="sparse",
target_size=(img_size,img_size))

In [24]:
train_image, train_labels = train_generator.next()

In [12]:

from tensorflow.keras.applications import MobileNet

In [25]:

vgg16 = tf.keras.applications.vgg16.VGG16(input_shape=(224, 224, 3),
                                       weights='imagenet',
                                       include_top=False)

x = tf.keras.layers.Flatten()(vgg16.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

predictions = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=vgg16.inputs, outputs=predictions)

for layer in vgg16.layers:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [26]:
model.summary()

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_sparse_categorical_accuracy', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

history = model.fit(train_generator,steps_per_epoch=20, validation_data= valid_generator,
                           validation_steps=5,epochs=30,callbacks=[checkpoint])

In [53]:
test_generator.reset()
pred = model.predict_generator(test_generator,verbose=1,steps=test_generator.samples/16)

In [59]:
predicted_values=np.argmax(pred,axis=1)
labels = {0: 'Cataract', 1: 'Glaucoma', 2: 'Normal'}
pred_val = [labels[key] for key in predicted_values]

In [63]:
final_df = pd.DataFrame({'Filename': test_generator.filenames,'Prediction': pred_val})
final_df.tail()

In [75]:
for i in np.random.randint(1493,size=10):
    print(i)
    print('Filename: ',test_generator.filenames[i])
    print('Predicted: ', final_df.iloc[i]['Prediction'])
    print('Original: ',labels[test_generator.labels[i]])
    print('_______________________________________________________________________')